In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 27922451
paper_name = 'vlaming_leeuwen_2016' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [10]:
sheet_names = ['UpTag','DownTag']
original_data_list = []

In [11]:
for s in sheet_names:
    original_data = pd.read_excel('raw_data/elife-18919-fig2-data2-v2.xlsx', sheet_name=s)
    print('Original data dimensions: %d x %d' % (original_data.shape))
    original_data['orf'] = original_data['ORF'].astype(str)
    
    # Eliminate the underscore annotations
    original_data['orf'] = original_data['orf'].apply(lambda x: x.split('_')[0])
    
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data = original_data.loc[t,:]
    original_data['data'] = original_data['average_log2']
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    original_data = original_data.groupby(original_data.index).mean()
    print(original_data.shape)
    original_data_list.append(original_data)

Original data dimensions: 4245 x 8
                Unnamed: 0  Plate            ORF    Gene          TAG  \
index_input                                                             
4240         3_UPTAGDot1OE      3  DOT1OE_Plate3  DOT1OE  UPTAGDot1OE   
4241         2_UPTAGDot1OE      2  DOT1OE_Plate2  DOT1OE  UPTAGDot1OE   
4242         1_UPTAGDot1OE      1  DOT1OE_Plate1  DOT1OE  UPTAGDot1OE   
4243         4_UPTAGDot1OE      4  DOT1OE_Plate4  DOT1OE  UPTAGDot1OE   
4244         5_UPTAGDot1OE      5  DOT1OE_Plate5  DOT1OE  UPTAGDot1OE   

             average_log2  data2012_log2  data2013_log2     orf  
index_input                                                      
4240             3.945793       3.710275       4.148205  DOT1OE  
4241             3.959812       3.828659       4.080030  DOT1OE  
4242             4.018617       4.018617            NaN  DOT1OE  
4243             4.087152       3.876036       4.271282  DOT1OE  
4244             4.290028       4.120874       4.441414  

In [13]:
original_data1, original_data2 = original_data_list

In [14]:
original_data = original_data1.join(original_data2, how='outer', lsuffix='_1', rsuffix='_2')

In [17]:
original_data['data'] = original_data[['data_1','data_2']].mean(axis=1)

# Prepare the final dataset

In [18]:
data = original_data[['data']].copy()

In [19]:
dataset_ids = [5373]
datasets = datasets.reindex(index=dataset_ids)

In [20]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [21]:
data.head()

dataset_id,5373
data_type,value
orf,
YAL002W,-0.459974
YAL007C,-0.020857
YAL008W,0.249730
YAL009W,-0.212003
YAL011W,-0.424054


## Subset to the genes currently in SGD

In [22]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 21


In [23]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [24]:
data.head()

,dataset_id,5373
,data_type,value
gene_id,orf,
2,YAL002W,-0.459974
5,YAL007C,-0.020857
6,YAL008W,0.249730
7,YAL009W,-0.212003
9,YAL011W,-0.424054


# Normalize

In [25]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [26]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [27]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [28]:
data_all.head()


dataset_id           5373          
data_type           value    valuez
gene_id orf                        
2       YAL002W -0.459974 -1.792801
5       YAL007C -0.020857 -0.040973
6       YAL008W  0.249730  1.038514
7       YAL009W -0.212003 -0.803536
9       YAL011W -0.424054 -1.649498

# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [30]:
from IO.save_data_to_db3 import *

In [31]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 27922451...
Inserting the new data...


100%|██████████| 1/1 [00:06<00:00,  6.93s/it]

Updating the data_modified_on field...
